# EASY

1. There are three issues that include the words "index" and "Oracle". Find the call_date for each of them

In [ ]:
SELECT DATE_FORMAT(call_date, '%Y-%m-%d %H:%i:%s') as Call_date , Call_ref	
FROM Issue
where Detail LIKE '%index%' AND Detail LIKE '%Oracle%';

2. Samantha Hall made three calls on 2017-08-14. Show the date and time for each

In [ ]:
Select DATE_FORMAT(call_date, '%Y-%m-%d %H:%i:%s') as Call_date, first_name, last_name
From Caller as c
INNER JOIN Issue as i
ON  c.Caller_id = i.Caller_id
where first_name = 'Samantha' AND last_name= 'Hall' AND CAST(call_date AS DATE) = '2017-08-14'; 

3. There are 500 calls in the system (roughly). Write a query that shows the number that have each status.

In [ ]:
SELECT status, COUNT(*) AS Volume
FROM Issue
GROUP BY status;

4. Calls are not normally assigned to a manager but it does happen. How many calls have been assigned to staff who are at Manager Level?

In [ ]:
SELECT count(*) as mlcc
FROM Issue as i
INNER JOIN Staff as s
ON i.Assigned_to = s.Staff_code
INNER JOIN Level AS l
ON s.Level_code = l.Level_code
WHERE l.Manager = 'Y';

5. Show the manager for each shift. Your output should include the shift date and type; also the first and last name of the manager.

In [ ]:
SELECT DATE_FORMAT(Shift_date, '%Y-%m-%d') as Shift_date, Shift_type, first_name, last_name
FROM Shift as sh
INNER JOIN Staff as s
ON sh.Manager = s.Staff_code

# Medium

6. List the Company name and the number of calls for those companies with more than 18 calls.

In [ ]:
SELECT c.Company_name, COUNT(*) as cc
FROM Customer as c
INNER JOIN Caller ca
ON c.Company_ref = ca.Company_ref
JOIN Issue as I
ON ca.Caller_id = I.Caller_id
GROUP BY c.Company_name
HAVING cc > 18;

7. Find the callers who have never made a call. Show first name and last name

In [ ]:
SELECT c.first_name, c.last_name
FROM Caller as c
LEFT JOIN Issue as i
ON c.Caller_id = i.Caller_id
where i.Caller_id IS NULL;

8. For each customer show: Company name, contact name, number of calls where the number of calls is fewer than 5

In [ ]:
SELECT
	a.Company_name,
	b.first_name,
	b.last_name,
	a.nc
FROM
	(
		SELECT
			Customer.Company_name,
			Customer.Contact_id,
			COUNT(*) AS nc
		FROM
			Customer
			JOIN
				Caller
				ON (Customer.Company_ref = Caller.Company_ref)
			JOIN
				Issue
				ON (Caller.Caller_id = Issue.Caller_id)
		GROUP BY
			Customer.Company_name,
			Customer.Contact_id
		HAVING
			COUNT(*) < 5
	)
	AS a
	JOIN
		(
			SELECT
				*
			FROM
				Caller
		)
		AS b
		ON (a.Contact_id = b.Caller_id);


9. For each shift show the number of staff assigned. Beware that some roles may be NULL and that the same person might have been assigned to multiple roles (The roles are 'Manager', 'Operator', 'Engineer1', 'Engineer2').

In [ ]:
SELECT
	a.Shift_date,
	a.Shift_type,
	COUNT(DISTINCT role) AS cw
FROM
	(
		SELECT
			shift_date,
			shift_type,
			Manager AS role
		FROM
			Shift
		UNION ALL
		SELECT
			shift_date,
			shift_type,
			Operator AS role
		FROM
			Shift
		UNION ALL
		SELECT
			shift_date,
			shift_type,
			Engineer1 AS role
		FROM
			Shift
		UNION ALL
		SELECT
			shift_date,
			shift_type,
			Engineer2 AS role
		FROM
			Shift
	)
	AS a
GROUP BY
	a.Shift_date,
	a.Shift_type;


10. Caller 'Harry' claims that the operator who took his most recent call was abusive and insulting. Find out who took the call (full name) and when.

In [ ]:
SELECT
	Staff.First_name,
	Staff.Last_name,
	a.Call_date 
FROM
	(
		SELECT
			Issue.Caller_id,
			MAX(DATE_FORMAT(call_date, '%Y-%m-%d %H:%i:%s')) AS Call_date 
		FROM
			Issue 
			JOIN
				Caller 
				ON (Issue.Caller_id = Caller.Caller_id) 
		WHERE
			Caller.First_name = 'Harry' 
		GROUP BY
			Issue.Caller_id
	)
	AS a 
	JOIN
		Issue 
		ON (a.Caller_id = Issue.Caller_id 
		AND a.Call_date = Issue.Call_date) 
	JOIN
		Staff 
		ON (Issue.Taken_by = Staff.Staff_code)